# Web scraping con Selenium

Hay ocasiones en las que la descarga de la página/sitio web y su posterior procesamiento con BeautifulSoup no es suficiente. Esto ocurre cuando la página web es dinámica, es decir, cuando el contenido final de una página depende de ciertas acciones por parte del usuario. Puede ocurrir:

* que el sitio del que queremos extraer los datos es una SPA (_Single Page Application_), 
* esté protegido por un login (mucho cuidado con los Términos de Uso)
* que necesitamos, por ejemplo, interactuar con el sitio, ya sea rellenando formularios, pulsando algún botón o enlace.


<img src='./HTML_examples/peticiondinamicas.png' style='width: 400px;'/>

Beautiful Soup no es capaz de ejecuta el código (JavaScript) requerido para cargar el contenido final de la página. 
Normalmente es el propio navegador el que ejecuta dicho código y carga el contenido. Esto es lo que haremos utilizando la biblioteca [Selenium](http://www.seleniumhq.org/docs/). Esta librería sirve para la automatización de tareas que hacen uso de un navegador. Recomendamos revisar [la documentación de Selenium para Python](http://selenium-python.readthedocs.io/) para sacar el máximo partido a esta librería.



## Instalación

Para poder utilizar Selenium es necesario:

1. Instalar la librería Selenium. Podemos instalarla desde el propio notebook con los gestores de paquetes pip (python) o conda (Anaconda).
2. Descargar el/los drivers para controlar un nevagador. Durante este taller usaremos el driver de Chrome, aunque podremos utilizar otros:

    + Chrome: <https://sites.google.com/a/chromium.org/chromedriver/downloads>
    + Edge: <https://developer.microsoft.com/en-us/microsoft-edge/tools/webdriver/>
    + Firefox: <https://github.com/mozilla/geckodriver/releases>
    + Safari: <https://webkit.org/blog/6900/webdriver-support-in-safari-10/>
    


In [1]:
# Descomenta y ejecuta una de las dos alternativas (o ejecútalos desde una consola de comandos):
# 1. Instalar desde Anaconda
#!conda install -c conda-forge selenium

# 2. Instalar con pip
#!pip install -U selenium

## Ejemplo introductorio 

A modo de ejemplo, vamos a ver cómo saber el número de productos de la página del sitio  https://aslan-blue-planet.es/tienda-vegana/.

<!--rellenar el campo de número de productos por página -->

La filosofía de Selenium es diferente ya que, en este caso, lo que vamos a hacer es programar la navegación automática por un sitio web. Para ello, Selenium lanza un navegador que controlaremos desde Python.

Para lanzar el navegador:

In [2]:
from selenium import webdriver
from selenium.webdriver.support.ui import Select

In [3]:
url = 'https://aslan-blue-planet.es/tienda-vegana/'
driver = webdriver.Chrome('../seleniumDrivers/chromedriver')  
# Si usamos Firefox:
# driver = webdriver.Firefox(executable_path='../seleniumDrivers/geckodriver')
driver.get(url)

Una vez que tenemos el navegador abierto y la página cargada, podemos navegar por la página mediante el repertorio de instrucciones `find_*` que proporciona [Selenium](https://selenium-python.readthedocs.io/locating-elements.html):


Recordando la estructura del  producto::

    <div class="ci-product">
    ...
		<a class="ci-product-img-container" 
           href="https://aslan-blue-planet.es/tienda/vego-bio-chocolate-blanco-con-almendras/">
			<img width="240" heigh...>
        </a>
	</div>

In [3]:
productos = driver.find_elements_by_css_selector('div.ci-product')
print ("Número de productos: {}".format(len(productos)))

Número de productos: 24


En total hemos encontrado 24 elementos con tag `div` de la clase `ci-product`.

Posteriormente, cuando ya hemos recuperado la información que necesitamos, terminamos cerrando el navegador.

In [4]:
driver.quit()

## Interactuando con la página

Ahora vamos a interactuar con la página. Buscaremos el campo donde se puede indicar el número de productos por página y posteriormente lo rellenaremos con un valor (de los permitidos). Consultar la ayuda de cómo se [interactúa con la página en la documentación de Selenium](https://selenium-python.readthedocs.io/navigating.html).

Comenzamos lanzando el navegador:

In [5]:
url = 'https://aslan-blue-planet.es/tienda-vegana/'
driver = webdriver.Chrome('../seleniumDrivers/chromedriver')  
# Si usamos Firefox:
# driver = webdriver.Firefox(executable_path='../seleniumDrivers/geckodriver')
driver.get(url)

Buscamos el elemento:

<img src='./HTML_examples/numpp.png' style='width: 400px;'/>


Solo hay tres valores permitidos y la estructura de dicho elemento es:
    
    <select name="ppp" class="select wppp-select" onchange="this.form.submit()">
        <option selected="selected" value="24">24 productos por página</option>
        <option value="36">36 productos por página</option>
        <option value="48">48 productos por página</option>
    </select>

In [7]:
elemento = driver.find_element_by_css_selector('select.wppp-select')
campo = Select(elemento)

La propia librería incluye la propiedad `options` para recuperar todas las posibles opciones de un campo.

In [9]:
opciones = campo.options
opciones

[<selenium.webdriver.remote.webelement.WebElement (session="b79adba82a826d6590fa80eb2584d87f", element="0.6884831430208855-2")>,
 <selenium.webdriver.remote.webelement.WebElement (session="b79adba82a826d6590fa80eb2584d87f", element="0.6884831430208855-3")>,
 <selenium.webdriver.remote.webelement.WebElement (session="b79adba82a826d6590fa80eb2584d87f", element="0.6884831430208855-4")>]

Obtenemos como resultado 3 opciones. Si consultamos el texto:


In [10]:
valores = [e.text for e in opciones ]
valores

['24 productos por página',
 '36 productos por página',
 '48 productos por página']

Para seleccionauna opción de las permitidas, tenemos varias funciones:

    * campo.select_by_index(index)
    * campo.select_by_visible_text("text")
    * campo.select_by_value(value)

In [11]:
campo.select_by_index(2)
# campo.select_by_visible_text(valores[2])   # esta opción tambiés es válida

En este caso, después de rellenar el campo no es necesario pulsar ningún botón adicional. Se ejecuta el código necesario para   actualizar la página automáticamente. Si consultamos ahora los productos por página:

In [12]:
productos = driver.find_elements_by_css_selector('div.ci-product')
print ("Número de productos: {}".format(len(productos)))

Número de productos: 48


Ahora tenemos 48 producots por página como era de esperar. 
Finalmente cerramos el driver.

In [13]:
driver.quit()

Ahora, si lo hacemos todo a la vez, el resultado es imprevisible....
A veces, la carga de la página tarda unos segundos, y cuando vamos a buscar un determinado elemento, no se encuentra y nos da error. 

In [14]:
# 1. Lanzamos el driver
url = 'https://aslan-blue-planet.es/tienda-vegana/'
driver = webdriver.Chrome('../seleniumDrivers/chromedriver')  
# Si usamos Firefox:
# driver = webdriver.Firefox(executable_path='../seleniumDrivers/geckodriver')
driver.get(url)

# 2. Buscamos el elemento para seleccionar el número  de páginas
elemento = driver.find_element_by_css_selector('select.wppp-select')
campo = Select(elemento)
opciones = elemento.find_elements_by_tag_name('option')
campo.select_by_index(2)

# 3. Recuperamos el número de productos cargados
productos = driver.find_elements_by_css_selector('div.ci-product')
print ("Número de productos: {}".format(len(productos)))

# 4. Cerramos el driver
driver.quit()

Número de productos: 48


## Introducciendo valores en formularios

En este tercer ejemplo, haremos scraping sobre la página `MegustairalSuper.com`. A modo de ejemplo vamos a buscar cuántas recetas de lentejas hay en dicho sitio web. Se puede acceder a esta pagina a través de esta URL:

<https://www.megustairalsuper.com/canal-cocina/recetas-2/>

<img src='./HTML_examples/super.png' style='width: 600px;'/>

In [17]:
url = "https://www.megustairalsuper.com/canal-cocina/recetas-2/"

Como podemos ver, para aplicar el filtro tenemos que rellenar el formulario. Inspeccionamos el elemento del formulario que permite introducir un término de búsqueda. La estructura tiene el siguiente aspecto:

    <form class="search" id="recipes-search-form" role="search" 
                         action="https://www.megustairalsuper.com/canal-cocina/recetas-2/" 
                         method="get" data-request="https://www.megustairalsuper.com/wp-admin/admin-ajax.php">
			...
            
			<input name="recipe_keyword" class="search-input" id="recipe_keyword" type="text" 
                      placeholder="Introduce un término de búsqueda y pulsa enter" value="">
                      
			<button title="Buscar" class="search-submit" role="button" type="submit"><i class="fa fa-search"></i></button>
	</form>
    

Primero lanzamos el navegador:

In [18]:
driver = webdriver.Chrome('../seleniumDrivers/chromedriver')  # Optional argument, if not specified will search path.
driver.get(url)

Buscamos el elemento que vamos a rellenar y le damos valor:

In [19]:
campo_input = driver.find_element_by_css_selector('form.search input.search-input')

In [20]:
# comprobamos que hemos recuperado lo que esperábamos
campo_input.tag_name

'input'

Para rellenar el campo usamos el método `send_keys` con el texto que queramos, por ejemplo `lentejas`:

In [21]:
filtro = 'lentejas'
campo_input.send_keys(filtro)

Finalmente, buscamos el botón y lo pulsamos:

In [22]:
#Finalmente, buscamos el botón y lo pulsamos:
lupa = driver.find_element_by_css_selector('form.search button.search-submit')
if lupa.is_enabled():
    lupa.click()

Como podemos observar, __la url de la página no ha cambiado__. Solo se ha cargado más información en la página (gracias a javascript), pero manteniéndose la URL.
Ahora, si queremos ver las recetas de la página, inspeccionamos el elemento que representa cada receta para ver su estructura. 

Lo más importante:

    <div id="recipe-21202" class="vc_grid-item vc_clearfix vc_col-sm-3 vc_grid-item-zone-c-bottom vc_visible-item" 
         style="position: absolute; left: 370px; top: 0px;">
			...
		<h4 style="text-align: left">Crema Esaú (lentejas)</h4>
			...															
	</div>

Para recuperar todos los elementos de la página que representan una receta, usamos el método `find_elements_by_css_selector`:

In [31]:
recetas = driver.find_elements_by_css_selector('div.vc_grid-item')
print ("Número de recetas: {}".format(len(recetas)))
print('---------------------------------')

for item in recetas:
    nombre = item.find_element_by_tag_name('h4').text
    print(nombre)

Número de recetas: 12
---------------------------------
Sopa de ajo
Tarta de Santiago
Tarta de queso al horno
Tarta de Mondoñedo
Tarta de Magosto
Tortillos de S. Andres de Teixido
Tortilla estilo Betanzos
Tortilla de patatas guisada
Tortilla de grelos con chorizos
Torta gallega
Torradas de nadal
Vieiras al vino


O lo que es lo mismo:

In [24]:
titulos = driver.find_elements_by_css_selector('div.vc_grid-item h4')
print ("Número de recetas: {}".format(len(titulos)))
print('---------------------------------')

for name  in titulos:    
    print(name.text)

Número de recetas: 12
---------------------------------
Lentejas estofadas con chorizo
Lentejas de la abuela
Crema Esaú (lentejas)
Lentejas estofadas con verduras
Lentejas vegetales
Lentejas con tomate y cebolla
Ensalada de legumbres
Ensalada de lentejas con judías verdes
Ensalada de lentejas
Lentejas con berenjena
Lentejas de cuaresma
Lentejas guisadas con zanahorias


Por último cerramos el driver.

In [25]:
driver.quit()

Ahora todo a la vez:

In [27]:
# 1. Lanzar del navegador
url = "https://www.megustairalsuper.com/canal-cocina/recetas-2/"
driver = webdriver.Chrome('../seleniumDrivers/chromedriver')  # Optional argument, if not specified will search path.
driver.get(url)

# 2. Rellenar el formulario con un valor
filtro = 'lentejas'
campo_input = driver.find_element_by_css_selector('form.search input.search-input')
campo_input.send_keys(filtro)

# 3. Hacer click en la lupa
lupa = driver.find_element_by_css_selector('form.search button.search-submit')
if lupa.is_enabled():
    lupa.click()

# 4. Buscar los elementos h4 y contarlos
titulos = driver.find_elements_by_css_selector('div.vc_grid-item h4')
print ("Número de recetas: {}".format(len(titulos)))
print('---------------------------------')

for name  in titulos:    
    print(name.text)
    
# 5. Cerrar el navegador    
driver.quit()

Número de recetas: 0
---------------------------------


Upsss!!!!!

## Las esperas

Vemos que la ejecución no ha funcionado correctamente. Esto se debe a que hay tareas (como la generación del resultado) que tardan un tiempo en ejecutarse, por lo que tenemos que esperar a que terminen para poder extraer un resultado. Para ello hay que implementar esperas [(Waits, en Selenium)](https://selenium-python.readthedocs.io/waits.html) en nuestro código.


### Esperas con librería time

La primera y más sencilla es hacer una pausa con `time.sleep(n)` .


In [29]:
import time

In [30]:
# 1. Lanzar del navegador
url = "https://www.megustairalsuper.com/canal-cocina/recetas-2/"
driver = webdriver.Chrome('../seleniumDrivers/chromedriver')  # Optional argument, if not specified will search path.
driver.get(url)

# esperamos 5 segundos
time.sleep(5)  

# 2. Rellenar el formulario con un valor
filtro = 'lentejas'
campo_input = driver.find_element_by_css_selector('form.search input.search-input')
campo_input.send_keys(filtro)

# 3. Hacer click en la lupa
lupa = driver.find_element_by_css_selector('form.search button.search-submit')
if lupa.is_enabled():
    lupa.click()

# Esperamos 5 segundos antes de buscar el elemento
time.sleep(5)     

# 4. Buscar los elementos h4 y contarlos
titulos = driver.find_elements_by_css_selector('div.vc_grid-item h4')
print ("Número de recetas: {}".format(len(titulos)))
print('---------------------------------')

for name  in titulos:    
    print(name.text)
    
# 5. Cerrar el navegador    
driver.quit()

Número de recetas: 12
---------------------------------
Lentejas estofadas con chorizo
Lentejas de la abuela
Crema Esaú (lentejas)
Lentejas estofadas con verduras
Lentejas vegetales
Lentejas con tomate y cebolla
Ensalada de legumbres
Ensalada de lentejas con judías verdes
Ensalada de lentejas
Lentejas con berenjena
Lentejas de cuaresma
Lentejas guisadas con zanahorias


### Esperas explícitas de Selenium

Otra opción es utilizar las _esperas explícitas_ de Selenium, con las que pedimos al driver que ejecute un método repetidas veces (durante un periodo máximo de tiempo) hasta que se cumpla una determianda condición. Por ejemplo, las condiciones pueden ser:

* `presence_of_element_located` : que aparezca un determinado elemento, 
* `presence_of_all_elements_located`: que aparezca un conjunto de elementos,
* `visibility_of_element_located`: que el elemento sea visible,
* `element_to_be_clickable`:  que se pueda hacer click sobre un elemento, 
* `title_is`: que el título de la página sea uno concreto, etc.

En caso de que se sobrepase el tiempo máximo de espera entonces python lanza una excepción o error. 

Para hacer uso de las esperas de Selenium tenemos que importar las librerías:

In [31]:
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import TimeoutException

from selenium.webdriver.support import expected_conditions as EC  ## condiciones de esperas
from selenium.webdriver.common.by import By                       ## localización de elementos

In [35]:
# 1. Lanzar del navegador
url = "https://www.megustairalsuper.com/canal-cocina/recetas-2/"
driver = webdriver.Chrome('../seleniumDrivers/chromedriver')  # Optional argument, if not specified will search path.
driver.get(url)

# 2. Rellenamos condición de búsqueda
filtro = 'lentejas'
campo_input = driver.find_element_by_css_selector('form.search input.search-input')
campo_input.send_keys(filtro)

# 3. Hacemos click sobre la lupa
lupa = driver.find_element_by_css_selector('form.search button.search-submit')
if lupa.is_enabled():
    lupa.click()
    
# Aquí vienen las esperas ....    

try:
    titulos = WebDriverWait(driver, 10).until(
        EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div.vc_grid-item h4"))
    )
    print ("Número de recetas: {}".format(len(titulos)))
    print('---------------------------------')

    for name  in titulos:    
        print(name.text)
    
except TimeoutException:
    print("No se ha podido encontrar el elemento o la página ha tardado demasiado")
finally:
    driver.quit() 

Número de recetas: 12
---------------------------------
Lentejas estofadas con chorizo
Lentejas de la abuela
Crema Esaú (lentejas)
Lentejas estofadas con verduras
Lentejas vegetales
Lentejas con tomate y cebolla
Ensalada de legumbres
Ensalada de lentejas con judías verdes
Ensalada de lentejas
Lentejas con berenjena
Lentejas de cuaresma
Lentejas guisadas con zanahorias


##  Cargar más recetas ejecutando javascript

Para cargar más recetas tenemos que pulsar el botón `CARGAR MÁS`. Pero solo lo haremos si el elemento que representa el botón está visible. 

<img src='./HTML_examples/cargarmas.png' style='width: 600px;'/>

Inspeccionado dicho elemento, su estructura es la siguiente:

    <div class="vc_btn3-container vc_grid-btn-load_more vc_btn3-inline">
		<a title="Cargar más" 
           class="vc_general vc_btn3 vc_btn3-size-md vc_btn3-shape-rounded vc_btn3-style-flat vc_btn3-color-blue"
           style="display: inline-block;" 
           href="javascript:;">Cargar más</a>
	</div>
    
Vamos a cargar el navegador y seleccionar recetas aplicando el criterio de búsqueda `huevos`:

In [36]:
url = "https://www.megustairalsuper.com/canal-cocina/recetas-2/"
driver = webdriver.Chrome('../seleniumDrivers/chromedriver')  # Optional argument, if not specified will search path.
driver.get(url)

filtro = 'huevos'
campo_input = driver.find_element_by_css_selector('form.search input.search-input')
campo_input.send_keys(filtro)

lupa = driver.find_element_by_css_selector('form.search button.search-submit')
if lupa.is_enabled():
    lupa.click()  

Ahora buscamos el elemento con tag `a` correspondiente a `Cargar más`. 

In [37]:
mostrar_mas = driver.find_element_by_css_selector("a[href*='javascript:;']")
print(mostrar_mas.text)

CARGAR MÁS


La intuición nos dice que podemos hacer click sobre el botón ejecutando el método `click`:

In [ ]:
mostrar_mas.click()

El error nos dice que este elemento _no es clicklable_. 

Aquí, el elemento con tag `a ` tiene como valor de atributo `href` una llamada a código javascript que se activa y ejecuta cuando el usuario lo pulsa.

Así que debemos indicar al driver que ejecute el código javaScript asociado a dicho elemento con el método `execute_script` en lugar de hacer `click` directamente sobre el elemento.

In [39]:
driver.execute_script("document.querySelector( 'a[href*=\"javascript:;\"]').click();") 

In [40]:
recetas = WebDriverWait(driver, 10).until(
        EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div.vc_grid-item"))
    )
print ("Número de recetas: {}".format(len(recetas))) 

Número de recetas: 24


Para cargar todas las recetas, tenemos que pulsar varias veces en el botón. Hemos de tener en cuenta que cuando no queden recetas para mostrar, el botón de `Cargar más` desaparece o se hace invisible y la ejecución del script no tiene efecto.
Para solucionarlo, podemos lanzar una espera con la condición `visibility_of_element_located`. 

In [41]:
boton_cargar = WebDriverWait(driver, 10).until(
        EC.visibility_of_element_located((By.CSS_SELECTOR, "a[href*='javascript:;']")))
driver.execute_script("document.querySelector( 'a[href*=\"javascript:;\"]').click();") 

Vamos pues a ejecutar el click hasta que el botón desaparezca:

In [42]:
clicks = 0
try:
    # pulso el botón mientras esté visible
    while True:
        # espero 10 segundos a que el botón se haga visible
        boton_cargar = WebDriverWait(driver, 10).until(
            EC.visibility_of_element_located((By.CSS_SELECTOR, "a[href*='javascript:;']")))
     
        # mientras sea visible
        clicks += 1
        driver.execute_script("document.querySelector( 'a[href*=\"javascript:;\"]').click();") 
    
except TimeoutException:
    print("Ha desaparecido el botón de CARGAR MÁS")
finally:
    print('Se han realizado ' +  str(clicks) + ' clicks.' )
    
    recetas = WebDriverWait(driver, 10).until(
        EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div.vc_grid-item h4"))
    )
    print ("Número de recetas: {}".format(len(recetas)))        
    driver.quit() 

Ha desaparecido el botón de CARGAR MÁS
Se han realizado 43 clicks.
Número de recetas: 43


¿Demasiados clicks? Esto es raro....

En realidad no tengo que hacer tantos clicks, solo debería hacer click si el número de recetas ha cambiado.

Ejecutamos todo el código:

In [43]:
# 1. Lanzar el navegador
url = "https://www.megustairalsuper.com/canal-cocina/recetas-2/"
driver = webdriver.Chrome('../seleniumDrivers/chromedriver')  # Optional argument, if not specified will search path.
driver.get(url)

# 2. Escribir criterio de búsqueda
# -----------------------------
filtro = 'huevos'
campo_input = driver.find_element_by_css_selector('form.search input.search-input')
campo_input.send_keys(filtro)

lupa = driver.find_element_by_css_selector('form.search button.search-submit')
if lupa.is_enabled():
    lupa.click()  

time.sleep(10) # seconds    

# 3. Memorizamos el numero de recetas inicial
recetas = WebDriverWait(driver, 10).until(
        EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div.vc_grid-item"))
    )
num_recetas = len(recetas)
print('Número de recetas inicial ' + str(num_recetas))

# 4. Hacemos CARGAR MÁS
# ---------
clicks = 0
try:
    # 5. Busco el botón de cargar más
    boton_cargar = WebDriverWait(driver, 50).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "a[href*='javascript:;']")))
         
    # 6. Mientras sea visible, lo ejecuto
    while boton_cargar.is_displayed(): 
        clicks += 1
        driver.execute_script("document.querySelector( 'a[href*=\"javascript:;\"]').click();") 
        
        boton_cargar = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "a[href*='javascript:;']")))   
        
        
        # 7. Miro a ver si ha aumentado el número de recetas
        recetas =  driver.find_elements_by_css_selector('div.vc_grid-item')
        while len(recetas) == num_recetas:
            recetas = recetas = driver.find_elements_by_css_selector('div.vc_grid-item')
        num_recetas = len(recetas)
        print('Número de recetas ' + str(num_recetas))    
        

# -----------------------------------
except TimeoutException:
    print("El botón de CARGAR MÁS no existe")    
    
finally:
    # 8. Contar el número de recetas totales
    print()
    print("Ya no es visible el botón de CARGAR MÁS")
    print("---------------------------------------")
    print('Se han realizado ' +  str(clicks) + ' clicks' )
     
    recetas =  driver.find_elements_by_css_selector('div.vc_grid-item')
    print ("Número total de recetas: {}".format(len(recetas)))
    #driver.quit() 
            

Número de recetas inicial 12
Número de recetas 24
Número de recetas 36
Número de recetas 43

Ya no es visible el botón de CARGAR MÁS
---------------------------------------
Se han realizado 3 clicks
Número total de recetas: 43


## Combinando ambas librerías


Finalmente, podemos combinar el uso de Selenium con BeautifulSoup. Para ello hemos de tener en cuenta que cada uno de los elementos extraídos con `find_*` disponen de los atributos `innerHTML` y `outerHTML`, que contienen el texto HTML del elemento. Podemos procesar este HTML con Beautiful Soup si nos resulta más cómodo.

También cabe decir que el driver mismo tiene el atributo `page_source` que contiene el código html.

In [44]:
from bs4 import BeautifulSoup
import requests

Vamos a buscar la información de todas las recetas. Dicha información se obtiene en otra página web, cuya url se encuentra en un elemento de la receta.

Buscamos los enlaces de todas las recetas:

    <div id="recipe-22390" class="vc_grid-item vc_clearfix vc_col-sm-3 vc_grid-item-zone-c-bottom vc_visible-item" 
         style="position: absolute; left: 0px; top: 0px;">
			...
		<a href="https://www.megustairalsuper.com/recipe/orejas-de-carnaval/" 
           title="Orejas de carnaval" class="vc_gitem-link vc-zone-link"></a>
			...						
	</div>

Guardamos todos los enlaces en una lista y posteriormente abrimos la página de cada enlace y recuperamos infomación de la receta.    

In [45]:
len(recetas)

43

In [46]:
enlaces = []
for receta in recetas:
    # obtenemos el html contenido en cada receta del elemento de selenium
    soup = BeautifulSoup(receta.get_attribute('outerHTML'), 'lxml')
    # a partir de aquí lo tratamos con BS4
    link = soup.select_one("div.vc_grid-item  a")['href']
    enlaces.append(link)

# cerramos el driver
driver.quit()

In [47]:
len(enlaces)

43

In [48]:
enlaces

['https://www.megustairalsuper.com/recipe/orejas-de-carnaval/',
 'https://www.megustairalsuper.com/recipe/pastel-de-coliflor-gratinada/',
 'https://www.megustairalsuper.com/recipe/quesada-gallega/',
 'https://www.megustairalsuper.com/recipe/revuelto-de-guisantes-con-langostinos/',
 'https://www.megustairalsuper.com/recipe/salpicon-de-pescado/',
 'https://www.megustairalsuper.com/recipe/tarta-de-santiago/',
 'https://www.megustairalsuper.com/recipe/tarta-de-queso-al-horno/',
 'https://www.megustairalsuper.com/recipe/tarta-de-magosto/',
 'https://www.megustairalsuper.com/recipe/tortillos-de-s-andres-de-teixido/',
 'https://www.megustairalsuper.com/recipe/tortilla-estilo-betanzos/',
 'https://www.megustairalsuper.com/recipe/tortilla-de-grelos-con-chorizos/',
 'https://www.megustairalsuper.com/recipe/torta-gallega/',
 'https://www.megustairalsuper.com/recipe/nabos-a-la-crema/',
 'https://www.megustairalsuper.com/recipe/menestra-gallega/',
 'https://www.megustairalsuper.com/recipe/judias-ve

Ahora recuperamos la información de cada receta. Para cada enlace tenemos que descargar la página. Usamos la función que habíamos visto en el documetno anterior.


In [49]:
def descargarPagina(url):
    """
    Carga y  procesa el contenido de una URL usando la librería request
    Muestra un mensaje de error en caso de no poder cargar la página
    """
    # Realizamos la petición a la web
    req = requests.get(url)

    # Comprobamos que la petición nos devuelve un Status Code = 200
    statusCode = req.status_code
    # la petición ha ido bien
    if statusCode == 200:   

        # Pasamos el contenido HTML de la web a un objeto BeautifulSoup()
        soup = BeautifulSoup(req.text,"lxml")
        return True, soup
    
    # la petición ha ido mal     
    else:   
        return False, print ("ERROR {}".format(statusCode))

<img src='./HTML_examples/tortilla.png' style='width: 600px;'/>


Entre la información de cada receta encontramos la siguiente estructura. Encontramos el título, el tipo de plato, el origen, etc.

    <div class="vc_row wpb_row row recipe-single">
     ...
        <h1>Tortilla estilo Betanzos</h1>
    
        <ul class="wpurp-recipe-tags" style="list-style:none !important;">
            <li class="wpurp-recipe-tags-plato" style="list-style:none !important;line-height:1.5em !important;">
                ...
                <a href="https://www.megustairalsuper.com/plato/primer-plato/">Primer plato</a>            
            </li>
            <li class="wpurp-recipe-tags-tipo" style="list-style:none !important;line-height:1.5em !important;">
                ...
                 <a href="https://www.megustairalsuper.com/tipo/huevos/">Huevos</a>            
            </li>
            <li class="wpurp-recipe-tags-origen" style="list-style:none !important;line-height:1.5em !important;">
                  ...       
                 <a href="https://www.megustairalsuper.com/origen/dag/">DAG</a>           
            </li>
        </ul>
        ...
    </div>


Ahora tendremos que descargar cada una de las páginas y recuperar la información:

In [50]:
datos = []
for url in enlaces:
    # descargar la página
    exito, receta_soup = descargarPagina(url)
    
    # buscar el título de la receta
    nombre = receta_soup.select_one('div.recipe-single h1').text   
    
    # buscar el resto de características  
    plato = receta_soup.select_one('ul.wpurp-recipe-tags li.wpurp-recipe-tags-plato a').text   
    tipo = receta_soup.select_one('ul.wpurp-recipe-tags li.wpurp-recipe-tags-tipo a').text
    origen = receta_soup.select_one('ul.wpurp-recipe-tags li.wpurp-recipe-tags-origen a').text
    
    # guardar la información
    datos_receta = {'Título': nombre, 'Plato': plato, 'Tipo': tipo, 'Procedencia': origen}
    datos.append (datos_receta)


In [51]:
import pandas as pd
# creamos una tabla y la guardamos en un fichero excel
tabla = pd.DataFrame(datos)
tabla.to_excel('recetas.xlsx', index = False)

In [25]:
tabla

,Plato,Procedencia,Tipo,Título
0,Postre,DAG,Postres,Orejas de carnaval
1,Primer plato,DAG,Vegetales,Pastel de coliflor gratinada
2,Postre,DAG,Postres,Quesada gallega
3,Primer plato,DAG,Vegetales,Revuelto de guisantes con langostinos
4,Primer plato,DAG,Pescados,Salpicón de pescado
5,Postre,DAG,Postres,Tarta de Santiago
6,Postre,DAG,Postres,Tarta de queso al horno
7,Postre,DAG,Postres,Tarta de Magosto
8,Primer plato,DAG,Huevos,Tortillos de S. Andres de Teixido
9,Primer plato,DAG,Huevos,Tortilla estilo Betanzos


## Extra: Generación de contenidos mediante la interacción con una web

Selenium nos facilita la interacción con la web. Vamos a poner como ejemplo la interacción con una herramienta de generación de [contraseñas mediante htdigest](https://httpd.apache.org/docs/2.4/programs/htdigest.html) (<https://websistent.com/tools/htdigest-generator-tool/>). Vamos primeramente a rellenar el formulario:

In [24]:
from selenium import webdriver

url = "https://websistent.com/tools/htdigest-generator-tool/"
usuario = "miUsuario"

driver = webdriver.Chrome('../seleniumDrivers/chromedriver')  # Optional argument, if not specified will search path.
driver.get(url)

element = driver.find_element_by_id("uname")
element.send_keys(usuario)

Si vamos al navegador veremos que hemos rellenado el primer cambio del formulario. A continuación rellenamos el resto.

In [25]:
element = driver.find_element_by_id("realm")
element.send_keys("miRealm")

element = driver.find_element_by_id("word1")
element.send_keys("12345")

element = driver.find_element_by_id("word2")
element.send_keys("12345")

Finalmente, buscamos el botón y lo pulsamos:

In [26]:
driver.find_element_by_id("generate").click();

Vemos que al final de la página se ha generado un texto con el resultado de la ejecución. Vamos a quedarnos con él:

In [27]:
output = driver.find_element_by_id("output").text
print (output[output.find(usuario):])
driver.quit()

miUsuario:miRealm:f183ef39e2332c681a3702eee9f8a9ac


Probemos a ejecutarlo todo de una sola vez:

In [30]:
driver = webdriver.Chrome('../seleniumDrivers/chromedriver')  # Optional argument, if not specified will search path.
driver.get(url)

element = driver.find_element_by_id("uname")
element.send_keys(usuario)

element = driver.find_element_by_id("realm")
element.send_keys("miRealm")

element = driver.find_element_by_id("word1")
element.send_keys("12345")

element = driver.find_element_by_id("word2")
element.send_keys("12345")

driver.find_element_by_id("generate").click();

output = driver.find_element_by_id("output").text
print (output[output.find(usuario):])
driver.quit()

.


Vemos que la ejecución no ha funcionado correctamente. Esto se debe a que hay tareas (como la generación del resultado) que tardan un tiempo en ejecutarse, por lo que tenemos que esperar a que terminen para poder extraer un resultado. Para ello hay que implementar esperas ([Waits, en Selenium](http://selenium-python.readthedocs.io/waits.html)) en nuestro código.

La primera y más sencilla es hacer una pausa antes de ejecutar una instrucción:

In [33]:
import time
driver = webdriver.Chrome('../seleniumDrivers/chromedriver')  # Optional argument, if not specified will search path.
driver.get(url)

element = driver.find_element_by_id("uname")
element.send_keys(usuario)

element = driver.find_element_by_id("realm")
element.send_keys("miRealm")

element = driver.find_element_by_id("word1")
element.send_keys("12345")

element = driver.find_element_by_id("word2")
element.send_keys("12345")

driver.find_element_by_id("generate").click();

# Esperamos 2 segundos antes de buscar el elemento
time.sleep(2)

output = driver.find_element_by_id("output").text
print (output[output.find(usuario):])
driver.quit()

miUsuario:miRealm:f183ef39e2332c681a3702eee9f8a9ac


Otra opción es utilizar las esperas explícitas de Selenium, con las que pedimos al driver que ejecute una método de manera constante durante durante un periodo de máximo de tiempo hasta que nos devuelva un determinado valor. En caso de que se sobrepase el tiempo máximo de espera entonces lanza una excepción. En nuestro ejemplo, vemos que el resultado siempre aparece en el mismo elemento y que cuando pulsamos en el botón primero aparece el texto "Loading". Por tanto, vamos a esperar hasta que desaparezca ese texto para extraer el resultado correcto:

In [41]:
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import TimeoutException

driver = webdriver.Chrome('../seleniumDrivers/chromedriver')  # Optional argument, if not specified will search path.
driver.get(url)

element = driver.find_element_by_id("uname")
element.send_keys(usuario)

element = driver.find_element_by_id("realm")
element.send_keys("miRealm")

element = driver.find_element_by_id("word1")
element.send_keys("12345")

element = driver.find_element_by_id("word2")
element.send_keys("12345")

driver.find_element_by_id("generate").click();

try:
    # Esperamos como máximo 10 segundos mientras esperamos a que desaparezca el texto "Loading"
    WebDriverWait(driver, 10).until_not(lambda driver: driver.find_element_by_id("output").text.startswith("Loading"))

    output = driver.find_element_by_id("output").text
    print (output[output.find(usuario):])

except TimeoutException:
    print("No se ha podido generar el resultado o la página ha tardado demasiado")
    
finally:
    driver.quit()

miUsuario:miRealm:f183ef39e2332c681a3702eee9f8a9ac


## Extra: Yummly

Lo primero que tenemos que hacer es analizar detenidamente el DOM de la web para saber cómo hacer el scroll. En este caso no es sencillo, ya que no hay que hacer scroll sobre la ventana del navegador sino sobre un elemento del mismo. Además, no podemos hacer click en los botones de scroll. Sin embargo, otra cosa que nos permite Selenium es ejecutar scripts en el navegador. Esto es lo que vamos a utilizar para la extracción de los resultados:

In [ ]:
from selenium import webdriver
import time
url = "http://www.yummly.com/recipes?q=&allowedCuisine=cuisine^cuisine-spanish&noUserSettings=true"
driver = webdriver.Chrome('../seleniumDrivers/chromedriver')  # Optional argument, if not specified will search path.
driver.get(url)

time.sleep(5)

recipes = driver.find_elements_by_class_name("recipe-card")
print ("Número de recetas: {}".format(len(recipes)))

# Realizamos scroll ejecutando código javascript en el navegador
driver.execute_script('cookbook = document.getElementsByClassName("cookbook")[0];')
driver.execute_script('maxScroll = document.getElementsByClassName("RecipeGrid")[0].clientHeight;')
driver.execute_script('cookbook.scrollTo(0, maxScroll);')

time.sleep(5)

recipes = driver.find_elements_by_class_name("recipe-card")
print ("Número de recetas: {}".format(len(recipes)))

driver.quit()

La automatización de este scraping no es nada sencilla ya que la página usa AJAX de forma muy profusa, con grandes retardos, lo que nos perjudica a la hora de extraer la información. Una posible solución puede ser la siguiente:

In [46]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import TimeoutException
import time

def heightHasChanged(driver, lastHeight):
    """
    Comprueba si, tras hacer scroll, el tamaño del panel en el que están las recetas ha cambiado
    """
    new_height = driver.execute_script('return document.getElementsByClassName("RecipeGrid")[0].clientHeight;')
    return last_height!= new_height

url = "http://www.yummly.com/recipes?q=&allowedCuisine=cuisine^cuisine-spanish&noUserSettings=true"
driver = webdriver.Chrome('../seleniumDrivers/chromedriver')  # Optional argument, if not specified will search path.
driver.get(url)

last_height = 0

time.sleep(5)

try:
    while True:
        # Realizamos scroll ejecutando código javascript en el navegador
        driver.execute_script('cookbook = document.getElementsByClassName("cookbook")[0];')
        driver.execute_script('maxScroll = document.getElementsByClassName("RecipeGrid")[0].clientHeight;')
        driver.execute_script('cookbook.scrollTo(0, maxScroll);')

        # Esperamos hasta que se actualice el scroll o lleguemos hasta el final
        WebDriverWait(driver, 10).until(lambda driver: heightHasChanged(driver,last_height))
        
        last_height = driver.execute_script('return document.getElementsByClassName("RecipeGrid")[0].clientHeight;')

except TimeoutException:
    # Se supone que hemo hecho scroll hasta el final 
    recipes = driver.find_elements_by_class_name("recipe-card")
    print ("Número de recetas: {}".format(len(recipes)))

finally:
    driver.quit()

Número de recetas: 2221


Finalmente, podemos combinar el uso de Selenium con BeautifulSoup. Para ello hemos de tener en cuenta que cada uno de los elementos extraídos con `find_*` disponen de los atributos `innerHTML` y `outerHTML`, que contienen el texto HTML del elemento. Podemos procesar este HTML con Beautiful Soup, si nos resulta más cómodo:

In [61]:
from selenium import webdriver
import time
from bs4 import BeautifulSoup

url = "http://www.yummly.com/recipes?q=&allowedCuisine=cuisine^cuisine-spanish&noUserSettings=true"
driver = webdriver.Chrome('../seleniumDrivers/chromedriver')  # Optional argument, if not specified will search path.
driver.get(url)

time.sleep(5)

# Realizamos scroll ejecutando código javascript en el navegador
driver.execute_script('cookbook = document.getElementsByClassName("cookbook")[0];')
driver.execute_script('maxScroll = document.getElementsByClassName("RecipeGrid")[0].clientHeight;')
driver.execute_script('cookbook.scrollTo(0, maxScroll);')

time.sleep(5)

recipeContainer = driver.find_element_by_class_name("RecipeContainer")
html = BeautifulSoup(recipeContainer.get_attribute('outerHTML'), 'lxml')
driver.quit()

# A partir de aquí, lo tratamos con BS4
recipes = html.select(".recipe-card h2.card-title a")
print("Número de recetas: {}".format(len(recipes)))
for title in recipes:
    print(title.text)



Número de recetas: 50
Portuguese Fish Stew
...
Gazpacho
